In [1]:
from imutils import paths
from imutils.video import VideoStream
import imutils
import face_recognition
import time
import argparse
import pickle
import cv2
import os

In [2]:
def check_trained(file_input = "encodings.pickle"):
    output_files = os.listdir()
    if file_input in output_files:
        return True
    
if check_trained():
    print('[INFO] Trained encoding found !')

[INFO] Trained encoding found !


In [3]:
def train_model(detection_method = "hog", file_input = "encodings.pickle", path_directory = "dataset"):
    # grab the paths to the input images in our dataset
    print("[INFO] quantifying faces...")
    imagePaths = list(paths.list_images(path_directory))

    # initialize the list of known encodings and known names
    knownEncodings = []
    knownNames = []

    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        # extract the person name from the image path
        print("[INFO] processing image {}/{}".format(i + 1,
            len(imagePaths)))
        name = imagePath.split(os.path.sep)[-2]

        # load the input image and convert it from RGB (OpenCV ordering)
        # to dlib ordering (RGB)
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(rgb,
            model=detection_method)

        # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)

        # loop over the encodings
        for encoding in encodings:
            # add each encoding + name to our set of known names and
            # encodings
            knownEncodings.append(encoding)
            knownNames.append(name)

    # dump the facial encodings + names to disk
    print("[INFO] serializing encodings...")
    data = {"encodings": knownEncodings, "names": knownNames}
    f = open(file_input, "wb")
    f.write(pickle.dumps(data))
    f.close()
    
#train_model()

In [4]:

def test( file_input = "encodings.pickle", model = 'hog'):
    file_output = "output"
    # load the known faces and embeddings
    print("[INFO] loading encodings...")
    data = pickle.loads(open(file_input, "rb").read())

    # initialize the video stream and pointer to output video file, then
    # allow the camera sensor to warm up
    print("[INFO] starting video stream...")
    vs = VideoStream(src=0).start()
    writer = None
    time.sleep(2.0)

    # loop over frames from the video file stream
    while True:
        # grab the frame from the threaded video stream
        frame = vs.read()

        # convert the input frame from BGR to RGB then resize it to have
        # a width of 750px (to speedup processing)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(frame, width=750)
        r = frame.shape[1] / float(rgb.shape[1])

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input frame, then compute
        # the facial embeddings for each face
        boxes = face_recognition.face_locations(rgb, model= model)
        encodingss = face_recognition.face_encodings(rgb, boxes)
        landmarks = face_recognition.face_landmarks(rgb, boxes, model = "large")
        
        names = []

        # loop over the facial embeddings
        for encoding in encodingss:
            # attempt to match each face in the input image to our known
            # encodings
            matches = face_recognition.compare_faces(data["encodings"], encoding)
            name = "Unknown"
            
            # check to see if we have found a match
            if True in matches:
                # find the indexes of all matched faces then initialize a
                # dictionary to count the total number of times each face
                # was matched
                #print("[INFO] Found !")
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}

                # loop over the matched indexes and maintain a count for
                # each recognized face face
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number
                # of votes (note: in the event of an unlikely tie Python
                # will select first entry in the dictionary)
                name = max(counts, key=counts.get)

            # update the list of names
            names.append(name)
            
        #draw landmark
        for landmark in landmarks:
            for part in landmark:
                #get points
                for (x,y) in landmark[part]:
                    #print(points)
                    cv2.circle(frame, (int(x*r), int(y*r)), 1, (0, 255, 255), -1)
            
        # loop over the recognized faces
        for ((top, right, bottom, left), name) in zip(boxes, names):
            # rescale the face coordinates
            top = int(top * r)
            right = int(right * r)
            bottom = int(bottom * r)
            left = int(left * r)

            # draw the predicted face name on the image
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)


        # check to see if we are supposed to display the output frame to
        # the screen
        if 1 > 0:
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break

    # do a bit of cleanup
    cv2.destroyAllWindows()
    vs.stop()

    # check to see if the video writer point needs to be released
    if writer is not None:
        writer.release()

In [5]:
if __name__ == "__main__":
    if check_trained():
        test()
    else:
        train_med()
        

[INFO] loading encodings...
[INFO] starting video stream...
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
[INFO] Found !
